In [1]:
from neo4j import GraphDatabase

def get_neo4j_driver() -> GraphDatabase.driver:
    """Establishes and returns a Neo4j session for AuraDB."""
    neo4j_uri = 'neo4j+s://9d1381c2.databases.neo4j.io:7687'
    neo4j_user = 'Shirley'
    neo4j_password = 'Sxl19950312'
    return GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

# Call the function to get the driver
driver = get_neo4j_driver()

# Open a session
with driver.session() as session:
    # Verify connectivity within the session
    session.run("RETURN 1")

print("Connectivity verified successfully.")


from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd


# Define a Cypher query to retrieve the required data
query = """
MATCH (n)
RETURN n
"""

# Execute the query and process the results
with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]

# Converting Neo4j Data to Pandas DataFrame Format
df = pd.DataFrame(neo4j_data)

#print(df)

Connectivity verified successfully.


In [ ]:
query = """
// Create a recommendation engine node called "RecommendationEngine1" and set the allergens to avoid and a description.
MERGE (r:RecommendationEngine {name: 'RecommendationEngine1'})
ON CREATE SET r.description = 'This recommendation engine suggests products that do not contain Gluten and Casein.'
SET r.allergensToAvoid = ['Gluten', 'Casein']

// Finding products that don't contain the allergens you want to avoid
WITH r, r.allergensToAvoid AS allergensToAvoid
MATCH (p:Product)
WHERE NONE(a IN allergensToAvoid WHERE EXISTS {
    MATCH (p)-[:HAS_RECIPE]->(:Recipe)-[:USES_INGREDIENT]->(:Ingredient)-[:HAS_RESEARCH]->(:IngredientResearch)-[:HAS_ALLERGEN]->(:Allergen {name: a})
})
AND NONE(a IN allergensToAvoid WHERE EXISTS {
    MATCH (p)-[:HAS_RECIPE]->(:Recipe)-[:USES_INGREDIENT]->(:Ingredient)-[:HAS_RESEARCH]->(:IngredientResearch)-[:has_ingredients]->(:IngredientSource)-[:SOURCED_FROM]->(:T3IngredientResearch)-[:HAS_ALLERGEN_INFO]->(:Allergen {name: a})
})

// Create a "RECOMMENDED" relationship connecting the recommendation engine node to the recommended product.
MERGE (r)-[:RECOMMENDED]->(p)

// Return to recommended product details
RETURN r.name AS recommendation_engine, r.description AS description, p.name AS product, p.description AS product_description, p.image AS image
LIMIT 10

"""

In [ ]:
query = """
// Create a recommendation engine node called "RecommendationEngine2" and set the allergens to avoid and a description.
MERGE (r:RecommendationEngine {name: 'RecommendationEngine2'})
ON CREATE SET r.description = 'This recommendation engine suggests products that do not contain Yeast.'
SET r.allergensToAvoid = ['Yeast']

// Finding products that don't contain the allergens you want to avoid
WITH r, r.allergensToAvoid AS allergensToAvoid
MATCH (p:Product)
WHERE NONE(a IN allergensToAvoid WHERE EXISTS {
    MATCH (p)-[:HAS_RECIPE]->(:Recipe)-[:USES_INGREDIENT]->(:Ingredient)-[:HAS_RESEARCH]->(:IngredientResearch)-[:HAS_ALLERGEN]->(:Allergen {name: a})
})
AND NONE(a IN allergensToAvoid WHERE EXISTS {
    MATCH (p)-[:HAS_RECIPE]->(:Recipe)-[:USES_INGREDIENT]->(:Ingredient)-[:HAS_RESEARCH]->(:IngredientResearch)-[:has_ingredients]->(:IngredientSource)-[:SOURCED_FROM]->(:T3IngredientResearch)-[:HAS_ALLERGEN_INFO]->(:Allergen {name: a})
})

// Create a "RECOMMENDED" relationship connecting the recommendation engine node to the recommended product.
MERGE (r)-[:RECOMMENDED]->(p)

// Return to recommended product details
RETURN r.name AS recommendation_engine, r.description AS description, p.name AS product, p.description AS product_description, p.image AS image
LIMIT 10


"""

In [ ]:
query = """

// Ensure that the user node named "User 1" already exists.
MATCH (user1:User {name: 'User 1'})

// Find the SelectedProductsUser node associated with "User 1".
MATCH (user1)-[:SELECTED]->(sp:SelectedProductsUser {name: 'SelectedProductsUser1'})

// Find GlutenFreeAndCaseinFree node
MATCH (gfcf:GlutenFreeAndCaseinFree)

// Find the product associated with the GlutenFreeAndCaseinFree node that is among the products selected in "User 1".
WITH user1, sp, gfcf
MATCH (sp)-[:CONTAINS]->(p:Product)-[:GLUTENFREECASEINFREE_TO]->(gfcf)

// Create a recommendation engine node called "RecommendationEngine3".
MERGE (r:RecommendationEngine {name: 'RecommendationEngine3'})
ON CREATE SET r.description = 'This recommendation engine suggests gluten-free and casein-free products for User 1 based on their selected products with no more than 1400 total calories.'

// Create a "RECOMMENDED" relationship connecting the recommendation engine node to the recommended product.
MERGE (r)-[:RECOMMENDED]->(p)

// Create a "RECOMMENDED_FOR" relationship between the recommendation engine node and the target user.
MERGE (r)-[:RECOMMENDED_FOR]->(user1)

// Return to recommended product details
RETURN r.name AS recommendation_engine, r.description AS description, user1.name AS target_user, p.name AS product, p.description AS product_description, p.calories AS calories

"""

In [ ]:
query = """
// Find products associated with the "Chips" community, excluding products with "onion" in the name of the ingredient or in the study.
MATCH (c:Community {name: 'Chips'})<-[:COMMUNITY_TO]-(p:Product)-[:HAS_RECIPE]->(:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(ir:IngredientResearch)
WHERE NOT EXISTS {
    MATCH (p)-[:HAS_RECIPE]->(:Recipe)-[:USES_INGREDIENT]->(i2:Ingredient)-[:HAS_RESEARCH]->(ir2:IngredientResearch)
    WHERE toLower(i2.name) CONTAINS 'onion' OR toLower(ir2.ingredient_name) CONTAINS 'onion'
}

// Collection of Chips products without "onion"
WITH collect(distinct p) as products

// Delete the existing RecommendationEngine5 node and its relationships.
MATCH (oldR:RecommendationEngine {name: 'RecommendationEngine5'})
DETACH DELETE oldR

// Create a recommendation engine node called "RecommendationEngine5".
CREATE (newR:RecommendationEngine {name: 'RecommendationEngine5', description: 'This recommendation engine suggests Chips products that do not contain onion in their ingredients or ingredient research.'})

// Pass the newly created node and product list to the next query section
WITH newR, products

// Create a "RECOMMENDS_PRODUCT" relationship to connect the recommendation engine node to the product.
UNWIND products as p
MERGE (newR)-[rp:RECOMMENDS_PRODUCT]->(p)

// Return to recommended product details
RETURN newR.name AS recommendation_engine, newR.description AS description, [p in products | p.name] AS recommended_products
"""

In [ ]:
query = """
// Query for chips products with the keyword onion.
MATCH (c:Community {name: 'Chips'})<-[:COMMUNITY_TO]-(p:Product)-[:HAS_RECIPE]->(:Recipe)-[:USES_INGREDIENT]->(:Ingredient)-[:HAS_RESEARCH]->(ir:IngredientResearch)
WHERE toLower(ir.ingredient_name) CONTAINS 'onion'
RETURN p.name AS productname, ir.ingredient_name AS ingredientname

"""

In [ ]:
query = """
// Find products with HealthStarRating ratings greater than 3.5
MATCH (p:Product)-[:HAS_RATING]->(n:HealthStarRating)
WHERE toFloat(n.rating) > 3.5

// Collect eligible products
WITH collect(distinct p) as products

// Create a recommendation engine node called "RecommendationEngine4".
CREATE (newR:RecommendationEngine {name: 'RecommendationEngine4', description: 'This recommendation engine suggests products with a HealthStarRating higher than 3.5.'})

// Pass the newly created node and product list to the next query section
WITH newR, products

// Create a "RECOMMENDS_PRODUCT" relationship to connect the recommendation engine node to the product.
UNWIND products as p
MERGE (newR)-[rp:RECOMMENDS_PRODUCT]->(p)

// Return to recommended product details
RETURN newR.name AS recommendation_engine, newR.description AS description, collect(p.name) AS recommended_products

"""


In [ ]:
query = """
Another one
// Find the community node called "Chips".
MATCH (c:Community {name: 'Chips'})

// Find products associated with the Chips community, as well as ingredient research and flavour labels for these products.
MATCH (c)<-[:BELONGS_TO]-(p:Product)-[:HAS_RECIPE]->(:Recipe)-[:USES_INGREDIENT]->(:Ingredient)-[:HAS_RESEARCH]->(ir:IngredientResearch)
WHERE EXISTS(ir.flavor)

// Collect the number of products for each flavour.
WITH c, ir.flavor AS flavor, COUNT(p) AS productCount
ORDER BY productCount DESC

// Create a recommendation engine node called "RecommendationEngine5".
MERGE (r:RecommendationEngine {name: 'RecommendationEngine5'})
ON CREATE SET r.description = 'This recommendation engine suggests different flavors of Chips products and the number of products for each flavor.'

// Create a "RECOMMENDS_FLAVOR" relationship to connect the recommendation engine node to the flavour and set the product quantity attribute.
MERGE (r)-[rf:RECOMMENDS_FLAVOR]->(f:Flavor {name: flavor})
ON CREATE SET rf.productCount = productCount

// Create a "RECOMMENDED_FOR" relationship to connect the recommendation engine node to the target community.
MERGE (r)-[:RECOMMENDED_FOR]->(c)

// Return to recommended flavour details
RETURN r.name AS recommendation_engine, r.description AS description, c.name AS target_community, f.name AS flavor, rf.productCount AS product_count
ORDER BY rf.productCount DESC
"""